In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import pandas as pd
import talib as ta
import pandas_ta as pandas_ta
import numpy as np
import matplotlib.pyplot as plt
from backtesting import Strategy
import plotly.express as px
from datetime import datetime
import MetaTrader5 as mt5

import random
random.seed(42)

from backbone.utils.general_purpose import diff_pips

In [3]:
import itertools
import yfinance as yf
from datetime import datetime
import MetaTrader5 as mt5
import pandas as pd
import pytz

tickers = [
    # Stocks argentinos
    # 'MELI',
    # 'YPF', 
    # 'BBAR', 
    # 'BMA', 
    
    # # Stocks extrangeros
    'NVDAm',
    'AVGOm',
    'METAm',
    'AMDm',
    'TSLAm',
    'MSFTm',
    'GOOGLm', 
    # 'APPL', 
    'NKEm', 

    # # Indices
    'US500m',
    # '^DJI',
    # '^IXIC',
    # '^RUT',


    # # Criptos 
    # 'BTC-USD',
    'ETHUSDm', 
    'LTCUSDm', 
    
    # # Comodities
    # 'GC=F', # Gold
    'USOILm', # Crude oil
    'UKOILm', # Crude oil
    # 'SI=F', # Silver
    # 'HG=F', # Copper
    'XNGUSDm', # Natural Gas
]

intervals = [
    mt5.TIMEFRAME_D1,
    mt5.TIMEFRAME_H4,
    mt5.TIMEFRAME_H2,
    mt5.TIMEFRAME_M30,
    mt5.TIMEFRAME_M15,
]

parameter_combinations = list(itertools.product(
    tickers, intervals
))

symbols = {}

print("MetaTrader5 package author: ", mt5.__author__)
print("MetaTrader5 package version: ", mt5.__version__)

# Establecer conexión con el terminal de MetaTrader 5
if not mt5.initialize():
    raise Exception("initialize() failed, error code =", mt5.last_error())

# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
utc_from = datetime(2021, 1, 1, tzinfo=timezone)
utc_to = datetime(2024, 1, 1, tzinfo=timezone)

for ticker, interval in parameter_combinations:
    # Obtener las tasas históricas
    rates = mt5.copy_rates_range(ticker, interval, utc_from, utc_to)
    
    # Crear DataFrame con las tasas
    df = pd.DataFrame(rates)
    
    # Convertir el tiempo de segundos a formato datetime
    df['time'] = pd.to_datetime(df['time'], unit='s')


    # Renombrar columnas para el ticker principal
    df = df.rename(columns={
        'time': 'Date',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'tick_volume': 'Volume'
    }).set_index('Date')


    if ticker not in symbols.keys():
        symbols[ticker] = {}
        symbols[ticker][interval] = {}

    symbols[ticker][interval] = df


# Cerrar la conexión con MetaTrader 5
mt5.shutdown()

# Mostrar el DataFrame final

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288


True

In [4]:

max_start_date = None
intervals_start_dates = {}

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date

intervals_start_dates



{16408: Timestamp('2022-03-20 00:00:00'),
 16388: Timestamp('2022-03-20 20:00:00'),
 16386: Timestamp('2022-03-20 22:00:00'),
 30: Timestamp('2022-03-20 22:00:00'),
 15: Timestamp('2022-03-20 22:15:00')}

In [5]:
from strategies import TradePullback, CumRSI, EndOfMonth, ExtremeTradePullback
import itertools

strategies = [
    TradePullback, 
    EndOfMonth, 
    CumRSI,
    ExtremeTradePullback
]

experiments = parameter_combinations = list(itertools.product(
    tickers, intervals, strategies
))

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:

    start_date = intervals_start_dates[interval]

    bt_train = Backtest(
        symbols[ticker][interval].loc[start_date:], 
        strategy,
        commission=0.0001,
        cash=15_000, 
        margin=1/30
    )
    
    stats = bt_train.run()
    
    df_stats = pd.DataFrame({
        'strategy':[strategy.__name__],
        'ticker':[ticker],
        'interval':[interval],

        'return':[stats['Return [%]']],
        'drawdown':[stats['Max. Drawdown [%]']],
        'win_rate':[stats['Win Rate [%]']], 
        'sharpe_ratio':[stats['Sharpe Ratio']],
        'trades':[stats['# Trades']],
        'avg_trade_percent':[stats['Avg. Trade [%]']],
        'exposure':[stats['Exposure Time [%]']],
        'final_equity':[stats['Equity Final [$]']],
        'Duration':[stats['Duration']],

    })

    performance = pd.concat([performance, df_stats])
    

In [6]:
performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']

In [7]:

performance[
    (performance['return'] > 20)
].sort_values(by=['return', 'drawdown'], ascending=[False, True]).head(50)


,strategy,ticker,interval,return,drawdown,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd
0,TradePullback,TSLAm,15,115.642756,16.105325,65.008026,1.634880,623,0.138655,18.570524,32346.413378,648 days 10:30:00,7.180405
0,EndOfMonth,METAm,16408,80.178536,2.613257,100.000000,1.575513,7,9.797066,9.375000,27026.780409,648 days 00:00:00,30.681455
0,CumRSI,LTCUSDm,15,76.178706,25.248032,62.673484,0.764920,5476,0.011293,39.049524,26426.805913,651 days 01:45:00,3.017214
0,TradePullback,LTCUSDm,30,71.261949,18.678796,65.475071,1.091103,1063,0.055877,17.097053,25689.292396,651 days 02:00:00,3.815125
0,EndOfMonth,TSLAm,16386,69.603881,11.176454,75.000000,1.268460,12,5.044809,11.399405,25440.582098,648 days 10:00:00,6.227725
0,EndOfMonth,LTCUSDm,30,63.130081,18.644332,68.181818,0.781122,22,2.488614,20.412147,24469.512088,651 days 02:00:00,3.386020
0,CumRSI,TSLAm,30,60.195638,33.729568,64.201183,0.419619,676,0.076807,37.616429,24029.345761,648 days 10:30:00,1.784655
0,EndOfMonth,LTCUSDm,16408,56.503988,10.916096,66.666667,0.849259,15,3.347873,18.223583,23475.598218,652 days 00:00:00,5.176208
0,EndOfMonth,LTCUSDm,15,52.623723,19.204666,68.181818,0.683975,22,2.146851,20.344028,22893.558481,651 days 01:45:00,2.740153
0,EndOfMonth,TSLAm,30,49.311152,15.159922,69.230769,0.963274,13,3.456461,12.086186,22396.672733,648 days 10:30:00,3.252731


In [8]:
performance[
    (performance['return/dd'] > 1) & (performance['trades'] > 1)
].groupby(by=['strategy', 'interval'])['return'].mean()

strategy              interval
CumRSI                15          76.178706
                      30          43.840283
                      16386       28.716348
                      16388       29.440685
EndOfMonth            15          30.092636
                      30          33.134899
                      16386       30.176871
                      16388       23.443874
                      16408       35.195759
ExtremeTradePullback  15          28.958142
                      30          24.493101
                      16386       16.856887
                      16388       12.965053
                      16408        3.991652
TradePullback         15          37.274943
                      30          35.323414
                      16386       26.504172
                      16388       15.771433
                      16408       10.052164
Name: return, dtype: float64

In [9]:
bt_train = Backtest(
    symbols['USOILm'][15], 
    TradePullback,
    commission=0.0001,
    cash=15_000, 
    margin=1/30
)

stats = bt_train.run()

bt_train.plot(filename='./RsiBBands.html', resample=False)


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p1669', ...)